In [1]:
# To disable `warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)`
import warnings
warnings.filterwarnings('ignore')

## 5. Disciplined Geometric Programming

__Disciplined geometric programming (DGP)__ is an analog of DCP <u>for log-log convex functions</u>, that is, functions of positive variables that are convex with respect to the geometric mean instead of the arithmetic mean.

While DCP is a ruleset for constructing convex programs, DGP is a ruleset for __log-log convex programs (LLCPs)__, which are problems that are convex after the variables, objective functions, and constraint functions are replaced with their logs, an operation that we refer to as a log-log transformation.

<u>Every __geometric program (GP)__ and __generalized geometric program (GGP)__ is an LLCP</u>, but there are LLCPs that are neither GPs nor GGPs.

CVXPY lets you form and solve DGP problems, just as it does for DCP problems.
Note that to solve DGP problems, you must pass the option `gp=True` to the `solve()` method.

In [7]:
import cvxpy as cp

# DGP requires Variables to be declared positive via `pos=True`.
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)
z = cp.Variable(pos=True)

objective = x * y * z
constraints = [
    4 * x * y * z + 2 * x * z <= 10,
    x <= 2 * y,
    y <= 2 * x,
    z >= 1
]
prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve(gp=True)

print(f"""
Optimal value: {prob.value:f}
x: {x.value:f}
y: {y.value:f}
z: {z.value:f}
""")


Optimal value: 2.000000
x: 1.000000
y: 2.000000
z: 1.000000



### Log-log curvature

Just as every Expression in CVXPY has a __curvature__ (constant, affine, convex, concave, or unknown), every Expression also has a __log-log curvature__.

A function $f\colon D\subseteq\mathbb{R}^n_{\succ 0}\to\mathbb{R}$ is said to be __log-log convex__ if the function $F(u)=\log f(e^u)$ with domain $\{u\in\mathbb{R}^n\mid e^u\in D\}$ is convex.
The function $F$ is called the __log-log transformation__ of $f$. The function $f$ is __log-log concave__ if $F$ is concave, and it is __log-log affine__ if $F$ is affine.

Every log-log affine function (called a __monomial function__ in GP) has the form
$f(x) = cx_1^{a_1}x_2^{a_2}\dotsb x_n^{a_n}$,
where $x\in\mathbb{R}^n_{\succ 0}$, $a_i\in\mathbb{R}$, and $c\in\mathbb{R}_{>0}$.

A sum of monomials (called a __posynomial function__ in GP) is a log-log convex function.

![](figures/log-log_curvature.png)

CVXPY's log-log curvature analysis can flag Expressions as __unknown__ even when they are log-log convex or log-log concave.

Note that any log-log constant expression is also log-log affine, and any log-log affine expression is log-log convex and log-log concave.

The log-log curvature of an Expression is stored in its `.log_log_curvature` attribute.

In [8]:
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)

constant = cp.Constant(2.0)
monomial = constant * x * y
posynomial = monomial + (x ** 1.5) * (y ** -1)
reciprocal = posynomial ** -1
unknown = reciprocal + posynomial

constant.log_log_curvature, monomial.log_log_curvature, \
posynomial.log_log_curvature, reciprocal.log_log_curvature, \
unknown.log_log_curvature

('LOG-LOG CONSTANT',
 'LOG-LOG AFFINE',
 'LOG-LOG CONVEX',
 'LOG-LOG CONCAVE',
 'UNKNOWN')

You can also check the log-log curvature of an Expression by calling the methods `is_log_log_constant()`, `is_log_log_affine()`, `is_log_log_convex()`, `is_log_log_concave()`.

### Log-log curvature rules

When formulating a DGP problem, all __Constants__ should be elementwise positive, and all __Variables__ and __Parameters__ must be constructed with the attribute `pos=True`.

DGP analysis is exactly analogous to DCP analysis. It is based on a [library of atoms (functions)](https://www.cvxpy.org/tutorial/dgp/dgp-atoms) with known monotonicity and log-log curvature and a a single <u>composition rule</u>.

A function $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is __log-log convex__ if $f$ is a log-log convex function and for each $\text{expr}_i$ one of the following conditions holds:

- $f$ is increasing in argument $i$ and $\text{expr}_i$ is log-log convex.
- $f$ is decreasing in argument $i$ and $\text{expr}_i$ is log-log concave.
- $\text{expr}_i$ is log-log affine.

A function $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is __log-log concave__ if $f$ is a log-log concave function and for each $\text{expr}_i$ one of the following conditions holds:

- $f$ is increasing in argument $i$ and $\text{expr}_i$ is log-log concave.
- $f$ is decreasing in argument $i$ and $\text{expr}_i$ is log-log convex.
- $\text{expr}_i$ is log-log affine.

A function $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is __log-log affine__ if $f$ is an log-log affine function and each $\text{expr}_i$ is log-log affine.

If none of the three rules apply, the expression $f(\text{expr}_1,\text{expr}_2,\dotsc,\text{expr}_n)$ is marked as having __unknown__ curvature.

If an Expression satisfies the composition rule, we say that the Expression "__is DGP__." You can check whether an Expression is DGP by calling the method `is_dgp()`.

In [9]:
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)

monomial = 2.0 * constant * x * y
posynomial = monomial + (x ** 1.5) * (y ** -1)

assert monomial.is_dgp()
assert posynomial.is_dgp()

### DGP problems

If a problem follows the DGP rules, it is guaranteed to be an __LLCP__ and solvable by CVXPY.

The DGP rules require that the problem objective have one of two forms:

- Minimize (log-log convex)
- Maximize (log-log concave)

The only valid constraints under the DGP rules are

- log-log affine `==` log-log affine
- log-log convex `<=` log-log concave
- log-log concave `>=` log-log convex

You can check that a problem, constraint, or objective satisfies the DGP rules by calling `object.is_dgp()`.

CVXPY will raise an exception if you call `prob.solve(gp=True)` on a non-DGP problem.

In [11]:
# DGP requires Variables to be declared positive via `pos=True`.
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)
z = cp.Variable(pos=True)

objective = x * y * z
constraints = [
    4 * x * y * z + 2 * x * z <= 10,
    x <= 2 * y,
    y <= 2 * x,
    z >= 1
]
assert objective.is_log_log_concave()
assert all(constraint.is_dgp() for constraint in constraints)

prob = cp.Problem(cp.Maximize(objective), constraints)
assert prob.is_dgp()

# All Variables must be declared as positive for an Expression to be DGP.
w = cp.Variable()
objective = w * x * y
assert not objective.is_dgp()

prob = cp.Problem(cp.Maximize(objective), constraints)
assert not prob.is_dgp()

### DGP atoms

#### Infix operators
The infix operators `+`, `*`, `/` are treated as atoms.

- The operators `*` and `/` are __log-log affine__ functions.
- The operator `+` is __log-log convex__ in both its arguments.

#### Multiplications
- You can use the `cvxpy.matmul` atom to multiply two matrices.
- To multiply two arrays or matrices elementwise, use the `cvxpy.multiply` atom.
- To take the product of the entries of an Expression, use the `cvxpy.prod` atom.

#### Transpose
Transpose `expr.T` is a __log-log affine__ function.

#### Power
The power operator `expr**p` is equivalent to the function `cp.power(expr, p)`, which is a __log-log affine__ function.

#### Scalar functions

![](figures/dgp_scalar_functions.png)

#### Elementwise functions

![](figures/dgp_elementwise_functions.png)

#### Vector/matrix functions

![](figures/dgp_vector_matrix_functions.png)